# KnowMore Notebook Edition

## 1. Enter Ids of datasets of interest

In [1]:
#list_datasetids = ['60', '64', '16']
list_datasetids = ['60', '64', '65', '16', '61', '82', '89']

# 2. Run data processing jobs on osparc

In [2]:
import osparc
import time
from pathlib import Path
from osparc.api import FilesApi, SolversApi
from osparc.models import File, Job, JobInputs, JobOutputs, JobStatus, Solver
import os
import json
from zipfile import ZipFile
import shutil
import pandas as pd
print(osparc.__version__)

OSPARC_API_KEY = os.environ['OSPARC_API_KEY']
OSPARC_API_SECRET = os.environ['OSPARC_API_SECRET']

cfg = osparc.Configuration(
    username=OSPARC_API_KEY,
    password=OSPARC_API_SECRET,
)
print(cfg.host)

#Save user selected datasetid in json
current_folder = os.getcwd()
main_folder = os.path.dirname(current_folder)
asset_input_folder = os.path.join(main_folder, 'assets', 'INPUT_FOLDER')
local_input_folder = os.path.join(asset_input_folder, 'tmp', 'fake-uuid-for-sample-data', 'INPUT_FOLDER')
local_output_folder = os.path.join(main_folder, 'static', 'notebook-output')
input_file = {}
input_file["datasetIds"] = list_datasetids
input_json_save_file = os.path.join(local_input_folder, 'input.json')
with open(input_json_save_file, 'w+') as f:
    # this would place the entire output on one line
    # use json.dump(lista_items, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(input_file, f)

with osparc.ApiClient(cfg) as api_client:

    files_api = FilesApi(api_client)
    
    print("-------------------------------")
    print("Running Python Runner on osparc")
    
    input_file_path = os.path.join(local_input_folder, 'input.json')
    input_file1: File = files_api.upload_file(file=os.path.join(asset_input_folder, "main.zip"))
    input_file2: File = files_api.upload_file(file=input_file_path)

    solvers_api = SolversApi(api_client)
    solver: Solver = solvers_api.get_solver_release(
        "simcore/services/comp/osparc-python-runner", "1.2.0"
    )

    job: Job = solvers_api.create_job(
        solver.id,
        solver.version,
        JobInputs(
            {
                "input_2": input_file2,
                "input_1": input_file1,
            }
        ),
    )

    status: JobStatus = solvers_api.start_job(solver.id, solver.version, job.id)
    
    while not status.stopped_at:
        time.sleep(5)
        status = solvers_api.inspect_job(solver.id, solver.version, job.id)
        print("Solver progress", f"{status.progress}/100", flush=True)

    outputs: JobOutputs = solvers_api.get_job_outputs(solver.id, solver.version, job.id)

    print(f"Job {outputs.job_id} got these results:")
    for output_name, result in outputs.results.items():
        print(output_name, "=", result)

    results_file: File = outputs.results["output_1"]
    download_path: str = files_api.download_file(file_id=results_file.id)
    if os.path.isfile(download_path) or os.path.isdir(download_path):
        print("Unzipping outputs")
        if os.path.isdir(local_output_folder):
            shutil.rmtree(local_output_folder)
        os.makedirs(local_output_folder)
        with ZipFile(download_path, 'r') as zipObj:
            zipObj.extractall(local_output_folder)
        print("Unzipped at ", local_output_folder)

    print("-------------------------------")
    print("Running custome Matlab script on osparc")  
    if os.path.isdir(os.path.join(local_output_folder, 'matlab-input-folder')):
        matlab_plots = 1
        input_file_path = os.path.join(local_output_folder, 'matlab-input-folder.zip')
        input_file1: File = files_api.upload_file(file=input_file_path)

        solvers_api = SolversApi(api_client)
        solver: Solver = solvers_api.get_solver_release(
            "simcore/services/comp/mat-parser", "1.1.0"
        )

        job: Job = solvers_api.create_job(
            solver.id,
            solver.version,
            JobInputs(
                {
                    "input_1": input_file1,
                }
            ),
        )

        status: JobStatus = solvers_api.start_job(solver.id, solver.version, job.id)
        while not status.stopped_at:
            time.sleep(5)
            status = solvers_api.inspect_job(solver.id, solver.version, job.id)
            print("Solver progress", f"{status.progress}/100", flush=True)

        outputs: JobOutputs = solvers_api.get_job_outputs(solver.id, solver.version, job.id)

        print(f"Job {outputs.job_id} got these results:")
        for output_name, result in outputs.results.items():
            print(output_name, "=", result)

        results_file: File = outputs.results["output_1"]
        download_path: str = files_api.download_file(file_id=results_file.id)
        print(download_path)
        if os.path.isfile(download_path) or os.path.isdir(download_path):
            print("Unzipping outputs")
            with ZipFile(download_path, 'r') as zipObj:
                zipObj.extractall(local_output_folder) 
    else:
        matlab_plots = 0
        print('No tabular data applicable for these datasets')

0.4.3
https://api.osparc.io
-------------------------------
Running Python Runner on osparc
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 100/100
Job e0f9358f-b540-434f-9da

# 3. Visualize results

In [19]:
list_datasetIds = ['60', '64']
if list_datasetIds and set(list_datasetIds).issubset(['60', '64', '65']):
    print('good')

good


In [42]:
input_info_file = os.path.join(local_output_folder, 'matlab-input-folder', 'matlab_input.xlsx')
df = pd.read_excel(input_info_file)
list_datasetIds = list(df['datasetId'].values)
list_datasetIds = ['56', '60']
print(list_datasetIds)
if list_datasetIds and set(list_datasetIds).issubset([60, 64, 65]):
    print('good')

['56', '60']


In [9]:
def check_list(my_list, check_list):
    for item in my_list:
        if item in check_list:
            

continue
